In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
# import labels 
def get_labels (camera_number, weather):
    """
    returns a dataframe with pairs (filename, labels) for the given camera and weather
    """
    path ="../../CNR-EXT-Patches-150x150/LABELS/camera{}.txt".format(camera_number)
    weater_label = weather[0].upper()
    # print("weater_label ", weater_label)

    # read as a file of strings
    with open(path, 'r') as f:
        data = list(f.read().splitlines())
    
    # get labels 
    data = [[d.split(' ')[0].split("/")[-1], int(d.split(' ')[1])]  for d in data]
    # filter by weather
    data = [d for d in data if d[0][0] == weater_label]
    data = pd.DataFrame(data, columns=['filename', 'label'])
    data['label_name'] = data['label'].apply(lambda x: 'busy' if x == 1 else 'free')

    return data

In [33]:
def classify (model, path):
    """
    classify image at a given path
    params:
        model: trained model
        path: path to image
    return:
        class_id: class id of the image
        class_name: class name of the image
        confidence: confidence of the model
    """

    img = tf.keras.utils.load_img(path, target_size=(150, 150))    
    img_array = tf.keras.utils.img_to_array(img)
    #normalize array
    img_array = img_array / 255.
    img_array = tf.expand_dims(img_array, 0) 
    
    predictions = model.predict(img_array, 1)
    score = tf.nn.softmax(predictions[0])
    class_id = np.argmax(score)
    class_name = 'busy' if class_id == 1 else 'free'
    confidence = np.max(score)
    # print("class_id {} class_name {} confidence {}".format(class_id, class_name, confidence))
    return class_id, class_name, confidence

def load_model(batch_size, epochs):
    model_dir = "CNN_model/{}_epochs_{}_batch_classifier".format(epochs, batch_size)
    model = tf.keras.models.load_model(model_dir)
    return model

In [34]:
def classify_data (camera_number, weather, batch_size, epochs, maxnfiles):
    """
    classify all images in the given camera and weather
    params:
        camera_number: camera number
        weather: weather
    return:
        dataframe with pairs (filename, class_id, class_name, confidence)
    """
    dir = "../../PATCHES_PROCESSED/"+weather.upper()+"/camera"+str(camera_number)
    import glob
    # dir = '../../CNR-EXT-Patches-150x150/PATCHES/OVERCAST/2015-11-16/camera1'
    filenames = glob.glob(dir+"/*.jpg")[:maxnfiles]

    model = load_model(batch_size=batch_size, epochs=epochs)
    data = []
    i = 0
    for filename in filenames:
        if i % 50 == 0: print("classifying {} of {}".format(i, len(filenames)))
        class_id, class_name, confidence = classify(model, filename)
        f = filename.split("/")[-1]
        data.append([f, class_id, class_name, confidence])
        i+=1
    data = pd.DataFrame(data, columns=['filename', 'class_id', 'class_name', 'confidence'])
    return data

In [35]:
d = classify_data (1, 'sunny', batch_size=32, epochs=10, maxnfiles=100)

classifying 0 of 100
classifying 50 of 100


In [ ]:
labels = get_labels(1, 'sunny')

In [74]:
# compare
for f in d['filename']:
    if f in labels['filename'].values:
        print ("label classifier {}, real label {}".format(d[d['filename'] == f]['class_name'].values[0], labels[labels['filename'] == f]['label_name'].values[0]))

In [75]:
labels

,filename,label,label_name
0,S_2016-01-16_10.05_C01_222.jpg,0,free
1,S_2016-01-15_14.05_C01_222.jpg,1,busy
2,S_2015-12-10_16.11_C01_193.jpg,1,busy
3,S_2016-01-13_08.48_C01_195.jpg,0,free
4,S_2015-12-17_14.48_C01_192.jpg,1,busy
...,...,...,...
6925,S_2016-01-12_09.47_C01_266.jpg,1,busy
6926,S_2016-01-12_09.17_C01_196.jpg,1,busy
6927,S_2016-01-15_08.05_C01_187.jpg,0,free
6928,S_2015-12-10_12.11_C01_189.jpg,1,busy


In [76]:
d

,filename,class_id,class_name,confidence
0,S_2015-11-22_1110_C01_225.jpg,0,free,1.0
1,S_2016-01-18_1705_C01_264.jpg,0,free,1.0
2,S_2015-11-22_1440_C01_225.jpg,0,free,1.0
3,S_2016-01-16_1305_C01_189.jpg,0,free,1.0
4,S_2016-01-16_1035_C01_190.jpg,0,free,1.0
...,...,...,...,...
95,S_2015-12-17_0918_C01_221.jpg,0,free,1.0
96,S_2015-11-27_0810_C01_184.jpg,0,free,1.0
97,S_2016-01-13_1548_C01_608.jpg,0,free,1.0
98,S_2016-01-13_1018_C01_608.jpg,0,free,1.0
